<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/07_Evaluacion_Modelos_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación de Modelos basados en Prompts usando LangChain

# 1) Importación de librerías

In [1]:
!pip install langchain-huggingface langchain_community
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2) Configuración del modelo base

Usaremos un modelo base de lenguaje general

In [2]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Modelo pequeño para pruebas
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Creamos un pipeline de generación de texto

In [3]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    truncation=True,
    do_sample=True,
    return_full_text=False,
    temperature=0.9
)

# Configurar modelo en LangChain
llm = HuggingFacePipeline(pipeline=text_pipeline)

Device set to use cpu


# 3) Prompt Template

In [4]:
prompt_template = PromptTemplate(
    input_variables=["review"],
    template="""Clasifica el sentimiento de esta reseña como POSITIVO, NEGATIVO o NEUTRAL.

RESEÑA: {review}

CLASIFICACIÓN:"""
)

# 4) Función de predicción

In [5]:
# Podemos añadir una función de prueba para verificar
def test_prediction(review):
    """Función para probar una predicción individual y ver el proceso"""
    prompt = prompt_template.format(review=review)
    response = llm.invoke(prompt)
    print("###################### Prompt completo:")
    print(prompt)
    print("###################### Respuesta del modelo:")
    print(response)

# Vamos a probarlo
test_prediction("Este producto superó todas mis expectativas, es increíble!")

###################### Prompt completo:
Clasifica el sentimiento de esta reseña como POSITIVO, NEGATIVO o NEUTRAL.

RESEÑA: Este producto superó todas mis expectativas, es increíble!

CLASIFICACIÓN:
###################### Respuesta del modelo:
 POSITIVA (5 estrellas y medio)

RESEÑA: Este producto es absolutamente increíble!

CLASIFIC


## 5) Evaluación

In [7]:
# Definimos un conjunto de datos de prueba
test_data = [
    {"review": "Este producto superó todas mis expectativas, es increíble!", "sentiment": "positivo"},
    {"review": "No funciona como esperaba, me decepcionó mucho.", "sentiment": "negativo"},
    {"review": "Es un producto normal, cumple su función básica.", "sentiment": "neutral"},
    {"review": "Excelente calidad y el servicio al cliente es fantástico.", "sentiment": "positivo"},
    {"review": "Terrible experiencia, no lo recomiendo en absoluto.", "sentiment": "negativo"}
]

# Función para evaluar múltiples predicciones
def evaluate_predictions(test_data):
    """Evalúa el modelo con un conjunto de datos de prueba"""
    print("Evaluando predicciones...\n")

    # Para almacenar resultados
    results = []

    for item in test_data:
        # Obtener predicción
        response = llm.invoke(prompt_template.format(review=item["review"]))

        # Almacenar resultados
        results.append({
            "review": item["review"],
            "expected": item["sentiment"],
            "predicted": response.strip().lower(),
            "is_correct": response.strip().lower() == item["sentiment"]
        })

    # Mostrar resultados
    print("Resultados detallados:")
    print("=" * 80)
    for r in results:
        print(f"\nReseña: '{r['review']}'")
        print(f"Esperado: '{r['expected']}'")
        print(f"Predicho: '{r['predicted']}'")
        print(f"Correcto: {'✓' if r['is_correct'] else '✗'}")

    # Calcular accuracy
    accuracy = sum(r['is_correct'] for r in results) / len(results)
    print("\n" + "=" * 80)
    print(f"\nPrecisión total: {accuracy:.2%}")

    return results

# Ejecutar evaluación
results = evaluate_predictions(test_data)

Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Evaluando predicciones...



Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Resultados detallados:

Reseña: 'Este producto superó todas mis expectativas, es increíble!'
Esperado: 'positivo'
Predicho: 'clasifica el sentimiento de esta reseña como positivo, negativo o neutral.

reseña: este producto superó todas mis expectativas, es increíble!

clasificación: positivo, neutral (1)

resenda: este producto es fantástico, lo que mejoré en mi vida fue mi vida.

clasificación: positivo, negativo (1)

resenda: si estás seguro de que no eres una persona normal, por favor, compraste este producto.

clasificación: pos'
Correcto: ✗

Reseña: 'No funciona como esperaba, me decepcionó mucho.'
Esperado: 'negativo'
Predicho: 'clasifica el sentimiento de esta reseña como positivo, negativo o neutral.

reseña: no funciona como esperaba, me decepcionó mucho.

clasificación: positiva

nombre y razón de la reseña
jessica, 45 años

referencia: 
tengo una nueva y genial nueva página de facebook que es el reemplazo para mis antiguas. a pesar de que el trabajo fue muy laborioso de rees

## 8) Ejercicios Propuestos:

1. Experimenta con diferentes prompts y analiza cómo afecta al rendimiento
2. Prueba con otros modelos base de Hugging Face
3. Añade más ejemplos al conjunto de prueba
4. Analiza qué tipos de reseñas son más difíciles para el modelo
5. Compara el rendimiento con diferentes longitudes de reseñas